Shamir Secret Sharing:

- Enter the secret, s, and the minimum number of recipients, t, of n shares.

- Let p > max(s, n), with $\mathbb{F}_p$ as our finite field. 

- Generate random coefficients for a polynomial of degree t-1, f(x), with the y-intercept as the secret text.

- Compute n points on the polynomial, (x, f(x)), with $x \neq 0$.

- Give each participant a unique point.

- To reconstruct the secret, use Lagrange interpolation formula with t many points of interpolation, and evaluate it at f(0) mod p.

In [3]:
import pandas as pd 
import sagemath as sg 
import random as rd
import numpy as np
import plotly.express as px
from random import randint
from math import gcd
import gmpy2
from gmpy2 import mpz, is_prime, gcdext, invert
import sympy as sp
from sympy import isprime, factorint, mod_inverse
import numpy.polynomial.polynomial as poly

Encoding

In [ ]:
secret = 67   
min_reps = 5 #min # of people to fidn the y intercept '67'
n = 100

p = max(secret, n) + 1  
def getPrime(p):  #testing to see if p is prime and if it isnt then it will loop until p is prime.
    k = False #proposing k and k will be true when p is prime

    while k == False: #is p prime? 
        if isprime(p) == True: 
            k = True
        else:
            p = p +1
        
    return p

prime = getPrime(p)
print(f'We are using Z mod {prime} as our finite field.')


def polynomialMaker(t,prime): 
        #INPUT: t  is the mimimum recipients later on.  #INPUT: prime 
    polylist = [secret]  #saying that there is one element in the list which is the secret
    for i in range(t-1): #taking a rand integer between 0 and prime (gotten from getprime), 
        polylist = polylist + [randint(0,prime)]
    eq = poly.Polynomial(polylist) #look @ numpy's documentation for more info on this function
    return eq #OUTPUT: eq is the polynomial equation that will be used to encode the secret

encodingPoly = polynomialMaker(min_reps,prime) 
print('\nPolynomial:')
display(encodingPoly) 


def splitter(eq, n, prime): 
    #INPUT: polynomial "eq" w/ the sectret, 
    #INPUT: n is the number of total recipients
    #INPUT: prime is the prime number 
    points = pd.DataFrame(columns=['X', 'f(X)']) #empty data frame of points ()
    list = []
    for i in range(n): #every # between 0 and 1 and number of ppoeple.
        k = randint(1,n) #random int between 1 and number of ppl
        if k in list: 
            continue #to the return points
        else:
            list = list + [k] # appending 'a random int' to the list. 
            points.loc[i, 'X'] = k #calling a function to give us the ith row of X. k becomes the ith row of the column 'X'
            num = eq(k)  
            points.loc[i, 'f(X)'] = num 
    return points

ciphertext = splitter(encodingPoly, n, prime)
display(ciphertext)

We are using Z mod 101 as our finite field.

Polynomial:


Polynomial([67., 91., 25., 12., 74.], domain=[-1,  1], window=[-1,  1], symbol='x')

,X,f(X)
0,99,7120308163.0
1,5,48897.0
2,15,3793807.0
3,9,497173.0
4,77,2606958729.0
...,...,...
87,18,7848013.0
88,74,2224013013.0
89,28,45770183.0
97,16,4906739.0


Decoding:

In [1]:
def modinv(a, p): #modular inverse function
    return pow(a, p - 2, p) #

def lagrange_symbolic(df, p): #synpy allows you to work with variables without defining them.
    x = sp.Symbol('x') #defining the variable x
    xs = [int(v) for v in df['X']] # list/array of x values in the column X
    ys = [int(v) for v in df['f(X)']] # same thing for y
    n = len(xs) #n is the length of the list of function inputs

    poly = 0 #
    for i in range(n):
        xi, yi = xs[i], ys[i]
        term = yi
        numerator = 1
        denominator = 1
        Li = 1  #is our Y. When poly is 0, our Polynomial P(x) = 0
        for j in range(n): #looping through all x values
            if i != j: #if i is not equal to j
                xj = xs[j] #xj is the jth x value
                numerator *= (x - xj) #numerator is the product of (x - xj) for all j != i
                denominator *= (xi - xj) % p #denominator is the product of (xi - xj) for all j != i
        Li = numerator * modinv(denominator, p) #
        poly += term * Li #appending this to our polynomial
                        # return is out yi


    # Expand and reduce modulo p
    poly = sp.expand(poly) 
    coeffs = sp.Poly(poly, x, modulus=p).all_coeffs()  # highest degree first

    # Convert to ascending order
    return [int(c) for c in reversed(coeffs)]

shift_factor = 0

decodingPoly = lagrange_symbolic(ciphertext.head(min_reps+shift_factor), prime)
eq = poly.Polynomial(decodingPoly)
print(f'Lagrange Polynomial is {eq}')
print(f'\nSafetext was {(eq(0)%prime)}')


NameError: name 'ciphertext' is not defined